In [1]:
%matplotlib inline

In [2]:
import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np

from sklearn.metrics import mean_squared_error

# Загрузка данных

In [3]:
%%time

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

submission = pd.read_csv("submission.csv")

attr = pd.read_csv("attr.csv")

CPU times: total: 53.2 s
Wall time: 59.4 s


# Проверка коррелляции параметров

In [4]:
train.corr()

,ego_id,u,v,t,x1,x2,x3
ego_id,1.000000,0.001828,0.001840,0.001420,0.000228,-0.001061,-0.000929
u,0.001828,1.000000,0.218330,-0.065169,0.011669,0.001498,-0.064033
v,0.001840,0.218330,1.000000,-0.092008,-0.019556,-0.022957,-0.022212
t,0.001420,-0.065169,-0.092008,1.000000,-0.190351,-0.073750,-0.025470
x1,0.000228,0.011669,-0.019556,-0.190351,1.000000,0.678870,0.089360
x2,-0.001061,0.001498,-0.022957,-0.073750,0.678870,1.000000,0.136608
x3,-0.000929,-0.064033,-0.022212,-0.025470,0.089360,0.136608,1.000000


In [5]:
test.corr()

,ego_id,u,v,t,x1,x2,x3
ego_id,1.000000,0.001169,0.003036,-0.012366,0.000762,0.000752,0.003607
u,0.001169,1.000000,0.216931,-0.062228,0.011462,0.002271,-0.064409
v,0.003036,0.216931,1.000000,-0.088513,-0.020211,-0.022287,-0.021533
t,-0.012366,-0.062228,-0.088513,1.000000,-0.191695,-0.074891,-0.026805
x1,0.000762,0.011462,-0.020211,-0.191695,1.000000,0.681151,0.089145
x2,0.000752,0.002271,-0.022287,-0.074891,0.681151,1.000000,0.136583
x3,0.003607,-0.064409,-0.021533,-0.026805,0.089145,0.136583,1.000000


Видно, что и на тестовых и на тренировочных данных довольно сильно взаимосвязаны параметры x1 и x2. При этом заметна положительная коррелляция. 

In [6]:
attr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14930748 entries, 0 to 14930747
Data columns (total 7 columns):
 #   Column      Dtype
---  ------      -----
 0   ego_id      int64
 1   u           int64
 2   age         int64
 3   city_id     int64
 4   sex         int64
 5   school      int64
 6   university  int64
dtypes: int64(7)
memory usage: 797.4 MB


In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122280372 entries, 0 to 122280371
Data columns (total 7 columns):
 #   Column  Dtype  
---  ------  -----  
 0   ego_id  int64  
 1   u       int64  
 2   v       int64  
 3   t       float64
 4   x1      float64
 5   x2      float64
 6   x3      float64
dtypes: float64(4), int64(3)
memory usage: 6.4 GB
None


In [9]:
attr.head()

,ego_id,u,age,city_id,sex,school,university
0,0,227,68,-1,1,778293348,-1
1,0,45,38,237065842,1,82803468,238500268
2,0,142,60,237065842,1,196560139,-1
3,0,280,66,-1,2,963209731,720783270
4,0,41,18,-1,2,308862409,-1


In [27]:
from tqdm import tqdm


def calculate_diff_age(df, attrs):
    df["diffAge"] = np.nan
    ego_id_list = train["ego_id"].drop_duplicates().values
    for ego_id in tqdm(ego_id_list):
        train_ego_net = train[train["ego_id"] == ego_id]
        attr_ego_net = attr[attr["ego_id"] == ego_id]
    
        parameter = np.zeros_like(train_ego_net["x1"].values)

        for i, (u, v) in enumerate(zip(train_ego_net["u"], train_ego_net["v"])):
    
            age_u = attr_ego_net.loc[attr_ego_net["u"] == u, ["age"]]
            age_v = attr_ego_net.loc[attr_ego_net["u"] == v, ["age"]]
    
            if not (age_v.empty or age_u.empty):
                parameter[i] = np.absolute(np.subtract(age_u["age"].values, age_v["age"].values))
            else:
                parameter[i] = np.nan
        df.loc[df["ego_id"] == ego_id, "diffAge"] = parameter
    return df


  0%|          | 88/61786 [02:44<31:59:09,  1.87s/it]


KeyboardInterrupt: 